In [2]:
# Import necessary packages
import os
import numpy as np
import pylab as py
import matplotlib.pyplot as plt
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import csv

# Paths for isochrones and output
iso_dir = 'isochrones/'
output_dir = 'output_diagrams/'

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Estimation variables
star_index = 0      # Set which star in the CSV to analyze
num_top_predictions = 10  # Control how many top predictions to plot

extinction_values = np.arange(20, -2, -2)  # AKs values from 20 to 0

# Define static isochrone parameters
dist = 4500
evo_model = evolution.Baraffe15()
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawHosek18b()
filt_list = ['jwst,F162M', 'jwst,F182M', 'jwst,F200W', 'jwst,F356W', 'jwst,F405N']
filters = ['m_jwst_F162M', 'm_jwst_F182M', 'm_jwst_F200W', 'm_jwst_F356W', 'm_jwst_F405N']
metallicity = 0
level_ages = np.linspace(1, 10, 19) * 1e6  # Define age array
log_age_arr = np.log10(level_ages)

# Load sample magnitudes, skipping the header row
sample_mags = []
with open('../../../s284-no-errors.csv', mode='r') as file:
    csvFile = csv.reader(file)
    next(csvFile)  # Skip header row
    for lines in csvFile:
        sample_mags.append([float(x) for x in lines])

# Chi-square minimization function
def chi_square_reverse_model(iso_grid, sample_mags):
    results = []
    for i, iso in enumerate(iso_grid):
        for star in iso.points:
            chi_square = sum(((sample_mags[k] - star[filters[k]]) ** 2) / star[filters[k]] for k in range(len(sample_mags)))
            results.append([chi_square, star['mass'], 10 ** log_age_arr[i]])
    
    return sorted(results, key=lambda x: x[0])

# Loop through each extinction value, delete old isochrones, and generate diagrams
for AKs in extinction_values:
    # Clear previous isochrones
    for file in os.listdir(iso_dir):
        os.remove(os.path.join(iso_dir, file))
    
    # Generate isochrone grid
    instances = np.array([
        synthetic.IsochronePhot(log_age, AKs, dist, metallicity=metallicity,
                                evo_model=evo_model, atm_func=atm_func,
                                red_law=red_law, filters=filt_list,
                                iso_dir=iso_dir)
        for log_age in log_age_arr
    ])

    # Run chi-square minimization and get top results
    sorted_results = chi_square_reverse_model(instances, sample_mags[star_index])
    top_results = sorted_results[:num_top_predictions]

    # Create figure with two subplots side-by-side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    cmap = plt.get_cmap('coolwarm')

    # Color-magnitude diagram (CMD)
    for i, instance in enumerate(instances):
        color = cmap(i / (len(instances) - 1))
        ax1.plot(instance.points[filters[0]] - instance.points[filters[1]], 
                 instance.points[filters[1]], color=color)
    ax1.set_xlabel('F162M - F182M')
    ax1.set_ylabel('F182M')
    ax1.invert_yaxis()

    # Magnitude-magnitude diagram (MMD)
    for i, instance in enumerate(instances):
        color = cmap(i / (len(instances) - 1))
        ax2.plot(instance.points[filters[0]], instance.points[filters[1]], color=color)
    ax2.set_xlabel('F162M')
    ax2.set_ylabel('F182M')
    ax2.invert_yaxis()

    # Create colorbar for both diagrams
    norm = Normalize(vmin=min(level_ages), vmax=max(level_ages))
    sm = ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=[ax1, ax2])
    cbar.set_label('Age (millions of years)')
    cbar.set_ticks(level_ages)
    cbar.set_ticklabels([f'{age/1e6:.1f}' for age in level_ages])

    # Plot top results on CMD with color gradient and labels
    for j, (chi_square, mass, age) in enumerate(top_results):
        closest_age_idx = np.argmin(abs(level_ages - age))
        iso_instance = instances[closest_age_idx]
        mass_idx = np.argmin(abs(iso_instance.points['mass'] - mass))
        color = cmap(j / (num_top_predictions - 1))
        ax1.plot(iso_instance.points[mass_idx][filters[0]] - iso_instance.points[mass_idx][filters[1]],
                 iso_instance.points[mass_idx][filters[1]], 'o', color=color, label=f"Mass: {mass:.2f}, Age: {age/1e6:.1f} Myr")
        ax2.plot(iso_instance.points[mass_idx][filters[0]],
                 iso_instance.points[mass_idx][filters[1]], 'o', color=color, label=f"Mass: {mass:.2f}, Age: {age/1e6:.1f} Myr")

    # Plot reference star in both diagrams
    ax1.plot(sample_mags[star_index][0] - sample_mags[star_index][1], sample_mags[star_index][1], '*', markersize=10, label="Reference Star")
    ax2.plot(sample_mags[star_index][0], sample_mags[star_index][1], '*', markersize=10, label="Reference Star")

    # Legends and saving
    ax1.legend()
    ax2.legend()
    fig.suptitle(f"AKs = {AKs}")
    plt.savefig(os.path.join(output_dir, f'CMD_MMD_AKs_{AKs}.png'))
    plt.close()

FileNotFoundError: [Errno 2] No such file or directory: 'isochrones/'